In [208]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [209]:
data = pd.read_csv(r"E:\git\py-adult-data-analysis\data\adult.csv")
df = pd.DataFrame(data)
df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [210]:
df_cleaned = df.loc[(df != "?").all(axis=1)]
df_cleaned.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [211]:
df = df_cleaned
df["salary_check"] = df["salary_check"] = ((df["salary"] == "<=50K") & (df["salary K$"] <= 50)) | (
            (df["salary"] == ">50K") & (df["salary K$"] > 50))

# Check data correctness for salary and salary $K  
(df["salary_check"] == True).all()

df.sample(10)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
25965,25965,28,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,38,United-States,<=50K,20,True
23575,23575,43,Local-gov,Some-college,Married-civ-spouse,Other-service,Husband,White,Male,40,United-States,<=50K,29,True
8076,8076,50,Self-emp-inc,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,<=50K,41,True
2301,2301,45,Private,HS-grad,Separated,Sales,Not-in-family,White,Female,40,United-States,<=50K,36,True
4722,4722,24,Private,HS-grad,Never-married,Sales,Own-child,White,Male,40,Dominican-Republic,<=50K,37,True
1003,1003,23,Local-gov,Bachelors,Never-married,Prof-specialty,Own-child,White,Female,60,United-States,<=50K,42,True
16187,16187,37,Federal-gov,HS-grad,Never-married,Exec-managerial,Unmarried,Black,Female,40,United-States,<=50K,47,True
28654,28654,31,Private,HS-grad,Never-married,Farming-fishing,Not-in-family,White,Male,40,Ireland,<=50K,47,True
1585,1585,40,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,45,United-States,>50K,284,True
6988,6988,63,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,36,United-States,>50K,299,True


# Task 1
Print the count of men and women in the dataset.

In [212]:
df.sex.value_counts()

Male      20380
Female     9782
Name: sex, dtype: int64

# Task 2
Find the average age of men in dataset

In [213]:
average_age = df[(df["sex"] == "Male")].age.mean()
average_age

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [214]:
df["native-country"].value_counts(normalize=True)["Poland"] * 100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [215]:
df.groupby("salary").age.agg(["mean", "std"])

,mean,std
salary,,
<=50K,36.60806,13.464631
>50K,43.95911,10.269633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [216]:
education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
selection = df[~df.education.isin(education) & (df.salary == ">50K")]
selection.sample(10)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
14702,14702,41,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,>50K,249,True
32183,32183,44,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,>50K,298,True
22145,22145,36,Private,9th,Married-civ-spouse,Sales,Own-child,White,Male,50,United-States,>50K,291,True
15891,15891,73,Private,7th-8th,Married-civ-spouse,Sales,Husband,White,Male,30,United-States,>50K,93,True
8015,8015,34,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,55,United-States,>50K,73,True
17675,17675,68,Self-emp-not-inc,Some-college,Married-civ-spouse,Sales,Husband,White,Male,40,United-States,>50K,74,True
31110,31110,40,Private,Some-college,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,105,True
12939,12939,52,Self-emp-inc,Some-college,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,240,True
13265,13265,43,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,42,United-States,>50K,199,True
7494,7494,31,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,119,True


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [217]:
df.groupby("education").age.describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [218]:
df["marital-status"] = df["marital-status"].apply(lambda x: "Married" if x.startswith("Married") else "Not Married")

df.groupby(["marital-status", "salary"]).salary.count()

marital-status  salary
Married         <=50K      8016
                >50K       6440
Not Married     <=50K     14638
                >50K       1068
Name: salary, dtype: int64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [219]:
max_hours = df["hours-per-week"].max()
max_hours


99

In [220]:
(df["hours-per-week"] == max_hours).sum()

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [221]:
df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
0,0,39,State-gov,Bachelors,Not Married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39,True
1,1,50,Self-emp-not-inc,Bachelors,Married,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,True
2,2,38,Private,HS-grad,Not Married,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27,True
3,3,53,Private,11th,Married,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43,True
4,4,28,Private,Bachelors,Married,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25,True


In [222]:

workclass_mapper = {value: num for num, value in enumerate(df.workclass.unique())}

df.workclass.replace(workclass_mapper, inplace=True)

native_country_mapper = {value: num for num, value in enumerate(df["native-country"].unique())}

df["native-country"].replace(native_country_mapper, inplace=True)

education_mapper = {value: num for num, value in enumerate(df.education.unique())}

df.education.replace(education_mapper, inplace=True)

occupancy_mapper = {value: num for num, value in enumerate(df.occupation.unique())}

df.occupation.replace(occupancy_mapper, inplace=True)

df.sex.replace({
    "Male": 1,
    "Female": 0,
}, inplace=True)


In [223]:
df_corr = df[["age", "workclass", "education", "occupation", "sex", "native-country", "salary K$"]].corr()
df_corr

,age,workclass,education,occupation,sex,native-country,salary K$
age,1.000000,0.064147,0.009060,0.002072,0.081993,-0.005066,0.208203
workclass,0.064147,1.000000,-0.022917,-0.046620,0.005652,-0.012075,0.086385
education,0.009060,-0.022917,1.000000,0.056654,-0.001781,0.060493,-0.039488
occupation,0.002072,-0.046620,0.056654,1.000000,0.263619,0.018083,-0.064373
sex,0.081993,0.005652,-0.001781,0.263619,1.000000,-0.006269,0.182642
native-country,-0.005066,-0.012075,0.060493,0.018083,-0.006269,1.000000,-0.020267
salary K$,0.208203,0.086385,-0.039488,-0.064373,0.182642,-0.020267,1.000000
